In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1845,2024-06-08,Brasil Nbb,17:10,Franca,Flamengo,1.95,1.77,154.5,1.83,1.87,1.5,1.84,1.59,l4HtZWhf,0.512821,0.564972,0.546448,0.534759,0.077792,0.2,0.6,NaN,NaN,139.908,14.571320,0.104149,109.700,23.467339,0.213923,104.890,109.700,119.68,149.72,0.0,0.0,0.0,0.0,0.068430,0.015289,0.103077,-0.62,-0.124,-7.661290,0.876190,0.8,-0.076190,1.33,0.266,2.894737,0.824305,0.8,-0.024305
1846,2024-06-08,Itália Liga A,15:30,Virtus Bologna,Milano,1.61,2.33,155.5,1.87,1.89,-3.5,1.90,2.64,St7TdXwP,0.621118,0.429185,0.534759,0.529101,0.050303,0.6,0.6,NaN,NaN,124.494,24.780207,0.199047,138.518,36.325121,0.262241,117.654,147.426,125.25,190.92,0.0,0.0,0.0,0.0,0.258435,0.007522,0.230511,0.20,0.040,15.250000,0.794127,0.8,0.005873,0.17,0.034,39.117647,0.651387,0.6,-0.051387
1847,2024-06-08,Argentina Liga A,21:00,Quimsa,Peñarol,1.19,4.40,160.5,1.87,1.79,-10.5,1.94,9.00,UXcNEeat,0.840336,0.227273,0.534759,0.558659,0.067609,0.8,0.0,NaN,NaN,108.028,12.645502,0.117058,NaN,NaN,NaN,110.432,0.000,100.04,0.00,0.0,0.0,0.0,0.0,0.812098,0.030912,0.912646,-0.18,-0.036,-5.277778,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1848,2024-06-08,Austrália Nbl1 Norte,06:00,North Gold Coast Seahawks,Gold Coast Rollers,2.39,1.53,179.5,1.84,1.86,3.5,1.92,1.44,pEo9Rd3s,0.418410,0.653595,0.543478,0.537634,0.072005,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,206.04,0.0,0.0,0.0,0.0,0.310261,0.007644,0.202031,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1849,2024-06-08,Austrália Nbl1 Norte,04:00,Brisbane Capitals F,Southern District Spartans F,1.80,1.92,150.5,1.87,1.83,-1.5,1.92,2.09,fZ3DzSWo,0.555556,0.520833,0.534759,0.546448,0.076389,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,147.20,0.0,0.0,0.0,0.0,0.045620,0.015289,0.059954,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1850,2024-06-08,Austrália Nbl1 Norte,04:30,South West Metro Pirates F,Mackay Meteorettes F,1.76,1.97,167.5,1.84,1.86,-1.5,1.87,1.96,pOdQXPG4,0.568182,0.507614,0.543478,0.537634,0.075796,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,192.64,0.00,0.0,0.0,0.0,0.0,0.079621,0.007644,0.033232,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1851,2024-06-08,Austrália Nbl1 Oeste,08:00,South West Slammers,Kalamunda Eastern Suns,1.72,2.02,166.5,1.85,1.85,-1.5,1.83,2.14,Mse8NR2o,0.581395,0.495050,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.113440,0.000000,0.110430,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1852,2024-06-08,Austrália Nbl1 Oeste,09:00,Perry Lakes Hawks,Goldfields Giants,1.73,2.01,177.5,1.83,1.85,-2.5,1.98,2.43,fXnLKPn4,0.578035,0.497512,0.546448,0.540541,0.075547,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,128.76,134.42,0.0,0.0,0.0,0.0,0.105877,0.007686,0.144308,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1853,2024-06-08,Eua Wnba,14:00,Connecticut Sun F,New York Liberty F,1.98,1.83,158.5,1.85,1.93,1.5,1.86,1.68,pKzJma7B,0.505051,0.546448,0.540541,0.518135,0.051499,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.055678,0.029930,0.071909,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1854,2024-06-08,Eua Wnba,18:00,Chicago Sky F,Atlanta Dream F,2.08,1.75,157.5,1.81,1.95,1.5,1.92,1.61,bNvNnJMH,0.480769,0.571429,0.552486,0.512821,0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,15:30,Itália Liga A,Virtus Bologna,Milano,155.5,1.87,1.0000,Over
1,21:00,Argentina Liga A,Quimsa,Peñarol,160.5,1.87,1.0000,Over
2,04:00,Austrália Nbl1 Norte,Brisbane Capitals F,Southern District Spartans F,150.5,1.87,1.0000,Over
3,08:00,Austrália Nbl1 Oeste,South West Slammers,Kalamunda Eastern Suns,166.5,1.85,0.9988,Over
4,09:00,Austrália Nbl1 Oeste,Perry Lakes Hawks,Goldfields Giants,177.5,1.83,1.0000,Over
5,14:30,Turquia Superliga,Anadolu Efes,Fenerbahce,168.5,1.87,1.0000,Over
